<br>
<br>

# Integrating Predictive Metrics in Psychological Research

Application of cross-validation methods to the data from Klein et al. (2014), **Investigating Variation in Replicability A ‘‘Many Labs’’ Replication Project**, accessible here: https://osf.io/wx7ck/ 

In [13]:
# Load required packages 
library(OutR2) # Predictive metrics
library(haven) # Read .sav data
library(ggplot2) # Plots
library(ggpubr) # Plots
library(grid) # Plots

### 1. Load data

In [14]:
path <- "C:/Users/DI.5057511/OneDrive - UAM/Tesis_Diego/Study 1/Code/Many_Labs_Data/" 
data <- as.data.frame(read_sav(paste(path,                             # File path
                                     "CleanedDataset.sav", sep = ""))) # Data

### 2. Data pre-processing

In the original study, 13 effects were analyzed (13 + 3, considering the four anchoring effects), of which 11 can be approached through regression analysis.

In [15]:
# 2. 1. Store in a list the data of each effect
# Effect
effect <- c("Sunk Cost", 
            "Anchoring NY", "Anchoring Chicago", "Anchoring MtEverest", "Anchoring Babies", 
            "Gamblers Fallacy", 
            "Quote Attribution", 
            "Flag Priming", 
            "Currency Priming", 
            "Imagined Contact", 
            "Math Attitudes")

# Experimental condition - group, Independent Variable IV
group <- c("sunkgroup", 
           "anch1group", "anch2group", "anch3group", "anch4group", 
           "gambfalgroup", 
           "quoteGroup", 
           "flagGroup", 
           "MoneyGroup", 
           "ContactGroup", 
           "sex")

# Dependent Variable, DV
DV <- c("sunkDV", 
        "anchoring1", "anchoring2", "anchoring3", "anchoring4", 
        "gambfalDV", 
        "quote", 
        "flagdv", 
        "Sysjust", 
        "Imagineddv", 
        "d_art")

# Generate list with 11 elements, one for each effect
data_sets <- vector(mode = "list", length = length(effect))
for(i in 1:length(effect)){
  
  # Get columns of each effect from the large data set
  col_loc <- c("sample", "referrer") # Location of data collection
  col_group <- group[i] # Experimental condition - group, IV
  col_DV <- DV[i] # Dependent variable, DV
  
  # Data set for each effect 
  data_effect <- data[, c(col_loc, col_group, col_DV)] # Get columns of each effect from the large dataset
  data_effect <- na.omit(data_effect) # Omit NA values
  data_effect <- data.frame(id = 1:nrow(data_effect), # Add ID
                            location = data_effect[, 1],
                            location_label = data_effect[, 2],
                            group = data_effect[, 3],
                            y = data_effect[, 4]) # Recode variable names
  if(group[i] == "sex") data_effect$group <- ifelse(data_effect$group == "m", 0, 1) # Recode as 0, 1
  
  # Add data set of each effect to the list
  data_sets[[i]] <- data_effect
  
}

# Add effect names to the list
names(data_sets) <- effect

# View list structure
lapply(data_sets, head)

### 3. Predictive metrics

In [16]:
# 3. 1. Conditions 
n_loc <- length(unique(data$sample)) # Number of locations
methodsR2 <- c("OOS", # Methods for estimating prediction error in R2 metric
               "Sample",
               "Ave_5k", "Ave_10k",
               "Pool_2k", "Pool_5k", "Pool_10k", "Pool_loo") 
methodsMSE <- c("OOS",
                "Sample",
                "RMSE_2k", "RMSE_5k", "RMSE_10k", "RMSE_loo") # Methods for estimating prediction error in MSE metric, here RMSE is reported to enhance the interpretability of the results
seed <- 10 # Set initial seed, for reproducible results

# 3. 2. Objects to store results
PredErrorR2 <- array(NA, dim = c(length(methodsR2), n_loc, length(data_sets)), # Store R2 
                     dimnames = list(methodsR2, 1:n_loc, names(data_sets)))
PredErrorRMSE <- array(NA, dim = c(length(methodsMSE), n_loc, length(data_sets)), # Store RMSE 
                       dimnames = list(methodsMSE, 1:n_loc, names(data_sets)))

# 3. 3. Compute predictive metrics
for(i in 1:length(data_sets)){ # For each effect in 11 effects
  
  data_set <- data_sets[[i]] # Data for the ith effect
  name <- names(data_sets)[i] # Effect
  Pop_Approx <- data_set # Real data, N =~ 5k as population approximation
  Y <- Pop_Approx$y # Y in Population Approximation
  
  for(j in 1:n_loc){ # For each location in 36 locations
    
    # Seet seed
    seed <- seed + 1 # Unique identifiable seed for each data set and location
    set.seed(seed)
        
    #-----
    # Prediction error
    #-----
    # 3. 3. 1. Get sample to fit the model,
    # and Out-of-sample observations (OOS, test sample) to compute the prediction error:
    # Sample
    Sample <- data_set[data_set$location == j, ] # Sample, location j
    n <- nrow(Sample) # Sample size
    # Out-of-sample, OOS
    OOS <- data_set[data_set$location != j, ] # OOS, rest of the locations
    
    # 3. 3. 2. Fit the model:
    fit <- glm(formula = y ~ group, data = Sample)
    
    # 3. 3. 3. Predictive metrics:
    # In-sample
    y_sample <- Sample$y # Observed values/ Criterion
    yhat_sample <- predict(fit, Sample) # Fitted values/ Prediction
    # R2
    R2 <- rsq(y = y_sample, yhat = yhat_sample) 
    PredErrorR2["Sample", j, name] <- R2
    # RMSE
    RMSE <- sqrt( mean( (y_sample - yhat_sample)^2 ) )
    PredErrorRMSE["Sample", j, name] <- RMSE
    
    # Out-of-sample
    y_oos <- OOS$y # Criterion
    yhat_oos <- predict(fit, newdata = OOS) # Prediction
    # MSE
    MSE_out <- mean( (y_oos - yhat_oos)^2 )
    PredErrorRMSE["OOS", j, name] <- sqrt(MSE_out) # RMSE
    # MST
    MST_out <- mean( (y_oos - mean(y_sample))^2 )
    # R2
    R2_out <- (MST_out - MSE_out) / MST_out
    PredErrorR2["OOS", j, name] <- R2_out 
    
    
    # Cross-validation:
    # K-folds
    kfolds <- c(2, 5, 10)
    for(k in kfolds){
      
      # Get estimates
      get <- rsq_cv(data = Sample, glmfit = fit, k = k, balanced = T, J = 2)
      
      # R2 averaging over k folds
      if(k != 2) PredErrorR2[sprintf("Ave_%sk", k), j, name] <- get$R2_ave
      
      # RMSE 
      PredErrorRMSE[sprintf("RMSE_%sk", k), j, name] <- get$RMSE_Cv
      # R2 pooling over k folds
      PredErrorR2[sprintf("Pool_%sk", k), j, name] <- get$R2_Cv
      
    }
    
    # Leave-one-out
    get <- rsq_cv(data = Sample, glmfit = fit, k = n, balanced = T, J = 2)
    # MSE
    PredErrorRMSE["RMSE_loo", j, name] <- get$RMSE_Cv
    # R2 pooling over n folds
    PredErrorR2["Pool_loo", j, name] <- get$R2_Cv
    
  }
  
  
}

### 4. Results

In [17]:
# Prediction Error (Pe)
# 4. 1. R2 
# Mean
PeR2_mean <- apply(PredErrorR2, MARGIN = c(1,3), mean)
round(PeR2_mean, 3)

# Median
PeR2_median <- apply(PredErrorR2, MARGIN = c(1,3), median)
round(PeR2_median, 3)

# Variance - Standard Deviation
SD_R2 <- apply(PredErrorR2, MARGIN = c(1,3), sd)
round(SD_R2, 3)

In [18]:
# 4. 2. RMSE
# Mean
PeRMSE_mean <- apply(PredErrorRMSE, MARGIN = c(1,3), mean)
round(PeRMSE_mean, 3)

# Median
PeRMSE_median <- apply(PredErrorRMSE, MARGIN = c(1,3), median)
round(PeRMSE_median, 3)

# Variance - Standard Deviation
SD_RMSE <- apply(PredErrorRMSE, MARGIN = c(1,3), sd)
round(SD_RMSE, 3)

### 5. Plots

In [19]:
# 5. 1. Prepare data for plotting (dfp)
dfp_all <- data.frame() # Store data of all effects in a long format for plotting
for(i in 1:length(effect)){
    
  # Transform wide data into long data for each effect
  dfp_effect <- as.data.frame(PredErrorR2[,, effect[i]]) # Get R2 values 
  dfp_long <- reshape(dfp_effect, v.names = "R2", # Wide to long format
                      idvar = "Method", ids = row.names(dfp_effect),
                      times = names(dfp_effect), timevar = "Location",
                      varying = list(names(dfp_effect)), direction = "long",
                      new.row.names = NULL)
  dfp_long$Effect <- effect[i] 
  dfp_all <- rbind(dfp_all, dfp_long[, c("Effect", "Location", "Method", "R2")]) # Add values of each effect
  rownames(dfp_all) <- NULL
  
}

# View data for plotting 
head(dfp_all)

In [20]:
# 5. 2. Plots
# Effects and CV methods
effect_plot <- effect # Effects for plot
methods_plot <- c("Sample", "Pool_loo", "Pool_10k", "Ave_5k", "Ave_10k") # CV methods for plot

# Aesthetics 
# Color
col_fill <- rainbow(n = length(methods_plot), start = 0.01, end = 0.80) # Fill color
col_fill <- col_fill[c(1, 5, 4, 2, 3)] # Order to align with colors of previous figures
darker_col = function(color, x = 90){ # Function for darkening colors
  dark_color <- vector(length = length(color))
  for(i in 1:length(color)){
    dark_color[i] <- colorRampPalette(c(color[i], "black"))(100)[x]
  }
  return(dark_color)
} 
col_out <- darker_col(col_fill, 90) # Outline color

# Labels
legend_labels <- c( # Legend labels
  Sample = expression("In-Sample"),
  Ave_5k = expression(Averaging["5K"]),
  Ave_10k = expression(Averaging["10K"]),
  Pool_10k = expression(Pooling["10K"]),
  Pool_loo = expression(Pooling["LOO"]))
facet_labels <- c( # Facet labels - Titles
  "Sunk Cost" = "Sunk Costs",
  "Anchoring NY" = "Anchoring - NY", 
  "Anchoring Chicago" = "Anchoring - Chicago", 
  "Anchoring MtEverest" = "Anchoring - MtEverest", 
  "Anchoring Babies" = "Anchoring - Babies",
  "Gamblers Fallacy" = "Gambler's Fallacy",  
  "Quote Attribution" = "Quote Attribution",
  "Flag Priming" = "Flag Priming",
  "Currency Priming" = "Currency Priming", 
  "Imagined Contact" = "Imagined Contact",
  "Math Attitudes" = "Math Attitudes"
)

In [21]:
# Plots
boxplots <- vector(mode = "list", length = length(effect_plot))
names(boxplots) <- facet_labels
for(i in 1:length(effect_plot)){
  
  rows_plot <- dfp_all$Effect == effect_plot[i] & dfp_all$Method %in% methods_plot # Get rows to plot
  dfp <- dfp_all[rows_plot,] # Data for plot
  dfp$Method <- factor(dfp$Method, levels = methods_plot) # Set Methods as factor to plot in order
  dfp$Effect <- facet_labels[i]
  
  # Boxplot
  boxplots[[i]] <- ggplot(dfp, aes(x = Method, y = R2)) +
    # Zero
    geom_hline(yintercept = 0, linetype = "solid",
               color = "lightslategray", linewidth = 0.5, alpha = 0.3) +
    stat_boxplot(geom = "errorbar", width = 0.15, lwd = 1.25, show.legend = F) +
    geom_boxplot(aes(y = R2, fill = Method, colour = Method), lwd = 1.25, show.legend = T) +
    # y-axis limit
    coord_cartesian(ylim = c(PeR2_mean["OOS" , effect_plot[i]] - 0.25,
                             PeR2_mean["OOS" , effect_plot[i]] + 0.25)) +
    # True prediction error
    geom_hline(yintercept = PeR2_median["OOS" , effect_plot[i]],
               col = "slategray3", linewidth = 2, linetype = "dashed") +
    # Colors
    scale_fill_manual(values = col_fill, labels = legend_labels) +
    scale_color_manual(values = col_out, labels = legend_labels) +
    # Axis-labels
    labs(x = NULL, y = NULL) +
    # Axis text
    scale_x_discrete(labels = legend_labels) +
    # Title
    facet_grid(~ Effect) +
    # Theme
    theme_bw() +
    theme(
      # Size of axis text
      axis.text.x = element_text(size = 10),
      axis.text.y = element_text(size = 12),
      # Facet grid text
      strip.text.x = element_text(size = 16),
      # Legend
      legend.text.align = 0, # align to the left
      legend.title = element_text(size = 14),
      legend.text = element_text(size = 12),
      legend.key.height = unit(1.5, "cm"),
      legend.key.width = unit(0.6, "cm"),
      legend.position = ("top")
    )
}

In [22]:
# View individual plot: specify the name of the effect to be displayed in the plot 
options(repr.plot.width = 8, repr.plot.height = 8)
boxplots$`Anchoring - Babies` # Anchoring Babies 

In [23]:
# Arrange plots
# Anchoring - Babies, Gambler's Fallacy and Sunk Costs
boxplots_three <- list(boxplots[[which(facet_labels == "Anchoring - Babies")]],
                       boxplots[[which(facet_labels == "Gambler's Fallacy")]],
                       boxplots[[which(facet_labels == "Sunk Costs")]])
plot_three <- ggarrange(plotlist = boxplots_three,
                        common.legend = T,
                        nrow = 1, ncol = 3,
                        legend = "top")
plot_three <- annotate_figure(plot_three,
                              left = textGrob(expression("R"^2), rot = 90, vjust = 1, gp = gpar(cex = 2)),
                              bottom = textGrob("Method", gp = gpar(cex = 2)))
# View plot
options(repr.plot.width = 16, repr.plot.height = 7)
plot_three

In [24]:
# All effects
suppressMessages( 
    for(i in 1:length(effect)) boxplots[[i]] <- boxplots[[i]] + scale_x_discrete(labels = NULL) + theme(axis.ticks.x = element_blank())
    )
plot_all <- ggarrange(plotlist = boxplots,
                      common.legend = T,
                      nrow = 2, ncol = 6,
                      legend = "top")
plot_all <- annotate_figure(plot_all,
                            left = textGrob(expression("R"^2), rot = 90, vjust = 1, gp = gpar(cex = 2)),
                            bottom = textGrob("Method", gp = gpar(cex = 2)))
# View plot
options(repr.plot.width = 24, repr.plot.height = 12)
plot_all